In [2]:
import numpy as np
from utils.visual import *
import torch
from utils.dataset import HybridDataset

In [3]:
import os.path as osp
# dir_path=r"H:\test\0802_full\5"
dir_path=r"..\littletestset\5"
set_name="5"
frame_num = 20
out_dir=r"..\outputs"
device = torch.device("cuda:0")
hyb=HybridDataset(dir_path,360)
# Ri2w=hyb.load_Ri2w(osp.join(dir_path,"I2C.txt"))
shape=hyb.load_shape(osp.join(dir_path,"shape.txt"))
first_fram, pose_batch=hyb.load_gt_seq(osp.join(dir_path,"pose.txt"))
Gw2c_list,K_list,view_name_list=hyb.load_Gw2c(r"..\calibration.json")
# joints2d=hyb.load_joints2d( dir_path, ["data3"], 10000)


In [4]:
from models import *
from scipy.spatial.transform import Rotation as R

smpl = SMPL(SMPL_MODEL_DIR, create_transl=False).to(device)

In [5]:
pose_batch=torch.from_numpy(pose_batch).to(device)
shape=torch.from_numpy(shape).to(device)

output = smpl(betas=shape.repeat([pose_batch.shape[0], 1]),
              body_pose=pose_batch[:,6:],
              global_orient=pose_batch[:,3:6],
              transl=pose_batch[:,:3], return_vertices=False)
output=output.joints

In [6]:
proj_list = []
for i in range(len(view_name_list)):
    K_ori = K_list[i]
    K = np.zeros((K_ori.shape[0], K_ori.shape[1] + 1), dtype=np.float32)
    K[:, :-1] = K_ori
    proj = K.dot(Gw2c_list[i])
    #proj = K.dot(Gw2c_list[i])
    proj_list.append(proj)

In [7]:
def projection(output,view):
    joints3d=torch.ones((output.shape[0],1,output.shape[1]),dtype=torch.float32,device=device)
    joints3d=torch.cat([output.permute(0,2,1),joints3d],1)
    joints2d=torch.einsum("mn,snl->sml",torch.from_numpy(proj_list[view]).to(device),joints3d)
    joints2d=joints2d/joints2d[...,2:3,:]
    joints2d=joints2d.permute([0,2,1])
    return joints2d

In [8]:
joints2d_list=[]
for view in range(len(proj_list)):
    joints2d=projection(output,view)
    joints2d_list.append(joints2d)

# joints2d_list[0][0]

In [9]:
import json
json_file=dict()

keypoints=['nose','left_eye','right_eye','left_ear','right_ear','left_shoulder','right_shoulder','left_elbow','right_elbow','left_wrist',
           'right_wrist','left_hip','right_hip','left_knee','right_knee','left_ankle','right_ankle']
keypoints_map=dict()
for i,key in enumerate(keypoints):
    keypoints_map[key]=i
skeleton=[[16, 14],
   [14, 12],
   [17, 15],
   [15, 13],
   [12, 13],
   [6, 12],
   [7, 13],
   [6, 7],
   [6, 8],
   [7, 9],
   [8, 10],
   [9, 11],
   [2, 3],
   [1, 2],
   [1, 3],
   [2, 4],
   [3, 5],
   [4, 6],
   [5, 7]]
json_file["categories"]=[{"supercategory": "person","id": 1,"name": "person","keypoints":keypoints,"skeleton": skeleton}]
json_file['images']=[]
json_file["annotations"]=[]

In [30]:
from subprocess import run
import os
import os.path as osp
if not osp.exists(out_dir):
    os.mkdir(out_dir)
result_dir=osp.join(out_dir,set_name)
if not osp.exists(result_dir):
    os.mkdir(result_dir)

json_list = []
# here 1,10 means video data1 to data10
for num in range(1,len(proj_list)+1):
    video_name="data"+str(num)+".mp4"
    if video_name not in os.listdir(dir_path):
        continue

    bundle_dir=osp.join(result_dir, "data"+str(num))
    if not osp.exists(bundle_dir):
        os.mkdir(bundle_dir)
    img_dir = osp.join(bundle_dir, set_name + "_data" + str(num))
    if not osp.exists(img_dir):
        os.mkdir(img_dir)

    temp = json_file.copy()
    
    video_path=osp.join(dir_path,"data"+str(num)+".mp4")
    name_prefix=set_name + "%02d"%num
    unpack_cmd = 'ffmpeg -i ' + video_path + ' -r ' + str(frame_num) + ' -f image2 ' +img_dir+ '\\'+name_prefix+'%04d.jpg'
    # unpack_cmd = 'ffmpeg -i ' + video_path + ' -f image2 ' +img_dir+ '\\'+name_prefix+'%04d.jpg'
    run(unpack_cmd)
    os.chdir(out_dir)

    mul = (60//frame_num)
    first = first_fram//mul + 1
    for i in range(joints2d_list[num-1].shape[0]//mul):
        # index=i+first_fram
        index = i + first
        img_id=name_prefix+"%04d"%index
        img_name=img_id+".jpg"
        temp['images'].append({"file_name":img_name,"id":int(img_id),"height": 2160,"width": 4096})
        keypoints=[int(p) for p in joints2d_list[num-1][i][[13,15,16,17,18,8,9,4,5,0,1,10,12,6,7,2,3]].reshape(-1).cpu().numpy()]
        l_bound=int(torch.min(joints2d_list[0][i*mul,:,0]).cpu().numpy())
        r_bound=int(torch.max(joints2d_list[0][i*mul,:,0]).cpu().numpy())
        d_bound=int(torch.min(joints2d_list[0][i*mul,:,1]).cpu().numpy())
        u_bound=int(torch.max(joints2d_list[0][i*mul,:,1]).cpu().numpy())
        temp["annotations"].append({"num_keypoints": 17,"iscrowd": 0,"keypoints":keypoints,"image_id":int(img_id),"bbox":[r_bound,u_bound,l_bound,d_bound],"category_id": 1,"id":int(img_id),"area":(r_bound-l_bound)*(u_bound-d_bound)/2})

    with open(osp.join(bundle_dir,"person_keypoints_"+set_name + "_data" + str(num)+".json"),"w") as f:
        json.dump(temp,f) 
